In [1]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr
import seaborn as sn

In [2]:
df = df = pd.read_csv('dataset/matches_datacleaning.csv', index_col = 0)
pd.set_option('display.max_columns', None) # to visualize all the columns
df.info()


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 184890 entries, 0 to 186072
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          184890 non-null  object 
 1   tourney_name        184890 non-null  object 
 2   surface             184728 non-null  object 
 3   draw_size           184890 non-null  float64
 4   tourney_level       184890 non-null  object 
 5   tourney_date        184890 non-null  object 
 6   match_num           184890 non-null  float64
 7   winner_id           184890 non-null  float64
 8   winner_entry        25701 non-null   object 
 9   winner_name         184890 non-null  object 
 10  winner_hand         184874 non-null  object 
 11  winner_ht           48560 non-null   float64
 12  winner_ioc          184890 non-null  object 
 13  winner_age          182080 non-null  float64
 14  loser_id            184890 non-null  float64
 15  loser_entry         43927 non-null

## new database building

In [3]:
name = pd.concat([df['winner_name'], df['loser_name']]) 
#name.rename(columns={0: 'name'}, inplace = True)
name.drop_duplicates(inplace=True)
print(name)


new_df = pd.DataFrame(name)
new_df.set_index(name, inplace=True)

#new_df.index.names = ['name']

0              KEI NISHIKORI
1            DANIIL MEDVEDEV
3         JO-WILFRIED TSONGA
5              JEREMY CHARDY
8             ALEX DE MINAUR
                 ...        
185021      DILARA SULTANOVA
185560      MADELEINE JESSUP
185780            KATIE CODD
185781             AMY HUANG
185940       CHLOE HENDERSON
Length: 10098, dtype: object


In [4]:
col1=df[['winner_name','winner_hand']]
col2=df[['loser_name','loser_hand']]
col2.rename(columns={'loser_name': 'winner_name'}, inplace = True)
col2.rename(columns={'loser_hand': 'winner_hand'}, inplace = True)
name2 = col1.append(col2) 
name2.rename(columns={'winner_name': 'name','winner_hand':'hand'}, inplace = True)
name2.drop_duplicates(inplace=True)
print(name2)


                      name hand
0            KEI NISHIKORI    R
1          DANIIL MEDVEDEV    R
3       JO-WILFRIED TSONGA    R
5            JEREMY CHARDY    R
8           ALEX DE MINAUR    R
...                    ...  ...
185021    DILARA SULTANOVA    U
185560    MADELEINE JESSUP    U
185780          KATIE CODD    U
185781           AMY HUANG    U
185940     CHLOE HENDERSON    U

[10100 rows x 2 columns]


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [5]:
df['winner_hand'].isnull().sum()

16

In [6]:
WinnerId_name = df.groupby(['winner_name']).winner_hand.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
WinnerId_name = WinnerId_name[WinnerId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

print(WinnerId_name)


Series([], Name: winner_hand, dtype: int64)


In [7]:
WinnerId_name = df.groupby(['loser_name']).loser_hand.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
WinnerId_name = WinnerId_name[WinnerId_name == 0] # it contains the TourneyId values associated with more than one Tourneyname value

print(WinnerId_name)



loser_name
ADAM HORNBY                      0
AINIUS SABALIAUSKAS              0
ALEX DIAZ                        0
ALEX KNAFF                       0
BERUK TSEGAI                     0
BRUNO BRITEZ                     0
CHRISTOPHE THOLL                 0
DAC TIEN NGUYEN                  0
FILIPPO MOGGIAN                  0
GIANMARCO DE SANTIS              0
HADY HABIB                       0
HERNANDO JOSE ESCURRA ISNARDI    0
IGNACIO GARCIA                   0
ILLIA BILOBORODKO                0
ISABEL PASCUAL MONTALVO          0
JEROME KYM                       0
JOSE OLIVARES                    0
LINH GIANG TRINH                 0
MATTHEW FOSTER ESTWICK           0
MICHAEL OGDEN                    0
MINJONG PARK                     0
NICOLAS XIVILLER                 0
PETER BERTRAN                    0
RONG MA                          0
ROSS WATSON                      0
SEBASTIAN ARCILA                 0
SEBASTIAN DOMINGUEZ              0
SEBASTIAN QUIROS                 0
WILLIAM K

In [8]:
print(df[df.winner_name=='ANASTASIA KULIKOVA'])

                 tourney_id    tourney_name surface  draw_size tourney_level  \
13271   W-WITF-BLR-03A-2017      MINSK $15K    Hard       32.0            15   
16988   W-WITF-FIN-03A-2017   HELSINKI $15K    Hard       32.0            15   
16997   W-WITF-FIN-03A-2017   HELSINKI $15K    Hard       32.0            15   
17002   W-WITF-FIN-03A-2017   HELSINKI $15K    Hard       32.0            15   
24211   W-WITF-SWE-05A-2017  STOCKHOLM $15K    Hard       32.0            15   
...                     ...             ...     ...        ...           ...   
182507   W-ITF-POR-07A-2020    W15 SANTAREM    Hard       32.0            15   
182731   W-ITF-POR-11A-2020       W15 PORTO    Hard       32.0            15   
182742   W-ITF-POR-11A-2020       W15 PORTO    Hard       32.0            15   
182748   W-ITF-POR-11A-2020       W15 PORTO    Hard       32.0            15   
183732   W-ITF-TUN-07A-2020    W15 MONASTIR    Hard       32.0            15   

       tourney_date  match_num  winner_

In [9]:

LoserId_name = df.groupby(['winner_name']).loser_hand.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.winner_name.isin(LoserId_name.index)].groupby(['winner_name']).loser_hand.unique()



winner_name
AANU AYEGBUSI                   [R, U]
AARON ADDISON                   [L, R]
ABBIE MYERS                  [R, U, L]
ABHINAV SANJEEV SHANMUGAM    [L, R, U]
ABIGAIL DESIATNIKOV          [U, L, R]
                               ...    
ZOFIA PODBIOL                   [U, R]
ZORA HUDSON                     [U, R]
ZOZIYA KARDAVA               [U, R, L]
ZSOMBOR PIROS                [L, R, U]
ZUZANA ZLOCHOVA              [U, R, L]
Name: loser_hand, Length: 4132, dtype: object